In [1]:
"""
Load forecasting pipeline notebook (runnable script)
Sections:
 A) LightGBM pipeline for Germany (DE) load forecasting
 B) Multi-country baseline evaluation (Top 3 models: Persistence, LightGBM, SARIMAX)
 C) Probabilistic forecasting (Quantile LightGBM) for DE day-ahead price
 Usage: run in a Jupyter notebook or as a script. Requires the dataset
 located at /mnt/data/time_series_60min_singleindex.csv
"""
import os, json, math, warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
import joblib
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

In [2]:
import os, json, math, warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
import joblib
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

##2. Data collection

In [14]:
df = pd.read_csv('/Users/phionanamugga/Documents/coding/datascience/Energy_Projects/time_series_15min_singleindex.csv')

##3. Data preprocessing

In [15]:
df.head()

,utc_timestamp,cet_cest_timestamp,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,DE_load_actual_entsoe_transparency,...,HU_solar_generation_actual,HU_wind_onshore_generation_actual,LU_load_actual_entsoe_transparency,LU_load_forecast_entsoe_transparency,NL_load_actual_entsoe_transparency,NL_load_forecast_entsoe_transparency,NL_solar_generation_actual,NL_wind_generation_actual,NL_wind_offshore_generation_actual,NL_wind_onshore_generation_actual
0,2014-12-31T23:00:00Z,2015-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-12-31T23:15:00Z,2015-01-01T00:15:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-12-31T23:30:00Z,2015-01-01T00:30:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-12-31T23:45:00Z,2015-01-01T00:45:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.describe()

,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,DE_load_actual_entsoe_transparency,DE_load_forecast_entsoe_transparency,DE_solar_capacity,...,HU_solar_generation_actual,HU_wind_onshore_generation_actual,LU_load_actual_entsoe_transparency,LU_load_forecast_entsoe_transparency,NL_load_actual_entsoe_transparency,NL_load_forecast_entsoe_transparency,NL_solar_generation_actual,NL_wind_generation_actual,NL_wind_offshore_generation_actual,NL_wind_onshore_generation_actual
count,201598.000000,201598.000000,131378.000000,201352.000000,201406.000000,201598.000000,201502.000000,201598.000000,201502.000000,175199.000000,...,35859.000000,193021.000000,198794.000000,197182.000000,201598.000000,201598.000000,181382.000000,193733.000000,193733.000000,200901.000000
mean,7070.161704,7039.857678,33.947254,135.283656,729.651345,9808.396916,9759.333258,55492.589700,54791.506975,42378.044395,...,177.082384,77.224803,470.611662,489.717953,12578.456013,12809.253119,162.335138,986.695963,340.447829,656.203710
std,1403.676483,1357.674902,16.815854,199.645788,693.810792,1398.512250,1384.276280,10041.148871,9523.623592,4306.349752,...,268.936739,76.613078,97.290734,114.502520,2290.400411,2507.304378,334.727140,831.882376,283.602966,635.109461
min,615.200000,3731.650000,-149.990000,0.000000,0.000000,6129.490000,6500.050000,29158.120000,28675.410000,37248.000000,...,0.000000,0.000000,10.000000,9.000000,6474.000000,4333.890000,0.000000,0.000000,0.000000,0.000000
25%,5938.800000,5940.000000,23.960000,0.000000,160.315000,8703.340000,8666.115000,47099.780000,46962.280000,38810.000000,...,0.000000,14.510000,405.000000,415.000000,10779.000000,10948.245000,0.000000,301.760000,88.270000,168.370000
50%,7032.400000,7019.120000,33.000000,16.000000,492.000000,9809.285000,9755.595000,55117.400000,54749.695000,40941.000000,...,2.820000,50.860000,469.000000,478.000000,12447.045000,12560.645000,0.000000,751.250000,271.920000,439.200000
75%,8080.800000,7987.442500,43.010000,224.000000,1140.000000,10856.987500,10815.052500,64337.655000,62901.085000,46092.000000,...,297.605000,120.620000,545.000000,549.000000,14301.000000,14638.560000,146.000000,1516.560000,579.220000,975.730000
max,10836.000000,11667.590000,977.740000,1172.000000,2992.000000,13815.370000,13653.090000,77852.940000,76392.520000,50508.000000,...,1018.140000,435.320000,861.100000,1356.000000,19404.440000,28269.770000,2296.000000,4322.460000,935.830000,3781.950000


In [17]:
df.columns

Index(['utc_timestamp', 'cet_cest_timestamp',
       'AT_load_actual_entsoe_transparency',
       'AT_load_forecast_entsoe_transparency', 'AT_price_day_ahead',
       'AT_solar_generation_actual', 'AT_wind_onshore_generation_actual',
       'BE_load_actual_entsoe_transparency',
       'BE_load_forecast_entsoe_transparency',
       'DE_load_actual_entsoe_transparency',
       'DE_load_forecast_entsoe_transparency', 'DE_solar_capacity',
       'DE_solar_generation_actual', 'DE_solar_profile', 'DE_wind_capacity',
       'DE_wind_generation_actual', 'DE_wind_profile',
       'DE_wind_offshore_capacity', 'DE_wind_offshore_generation_actual',
       'DE_wind_offshore_profile', 'DE_wind_onshore_capacity',
       'DE_wind_onshore_generation_actual', 'DE_wind_onshore_profile',
       'DE_50hertz_load_actual_entsoe_transparency',
       'DE_50hertz_load_forecast_entsoe_transparency',
       'DE_50hertz_solar_generation_actual',
       'DE_50hertz_wind_generation_actual',
       'DE_50hertz_wind_

In [18]:
df.dtypes

utc_timestamp                            object
cet_cest_timestamp                       object
AT_load_actual_entsoe_transparency      float64
AT_load_forecast_entsoe_transparency    float64
AT_price_day_ahead                      float64
                                         ...   
NL_load_forecast_entsoe_transparency    float64
NL_solar_generation_actual              float64
NL_wind_generation_actual               float64
NL_wind_offshore_generation_actual      float64
NL_wind_onshore_generation_actual       float64
Length: 61, dtype: object

In [19]:
df.dtypes.unique()

array([dtype('O'), dtype('float64')], dtype=object)

In [20]:
print(df.dtypes.value_counts())

float64    59
object      2
Name: count, dtype: int64


In [21]:
df.value_counts()

Series([], Name: count, dtype: int64)

##4. Feature Engineering

In [24]:
def make_features(series_target, exog_df=None, lags=(1,24,168)):
    X = pd.df(index=series_target.index)
    X['hour'] = series_target.index.hour
    X['dow'] = series_target.index.dayofweek
    X['month'] = series_target.index.month
    X['is_weekend'] = (series_target.index.dayofweek >=5).astype(int)
    for lag in lags:
        X[f'lag_{lag}'] = series_target.shift(lag)
    X['roll_3_mean'] = series_target.shift(1).rolling(3).mean()
    X['roll_24_mean'] = series_target.shift(1).rolling(24).mean()
    X['roll_168_mean'] = series_target.shift(1).rolling(168).mean()
    if exog_df is not None:
        for c in exog_df.columns:
            X[c] = exog_df[c]
    return X

##5. Model training

In [ ]:
def train_lgbm(X_train, y_train, params=None, num_round=500):
    if params is None:
        params = {'objective':'regression', 'metric':'mae', 'boosting_type':'gbdt', 'learning_rate':0.05,
                  'num_leaves':64, 'feature_fraction':0.8, 'bagging_fraction':0.8, 'bagging_freq':5, 'seed':42}
    dtrain = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(params, dtrain, num_boost_round=num_round)
    return model

##Model evaluation

In [ ]:


def run_de_load_pipeline(df, country='DE'):
    target_col = f'{country}_load_actual_entsoe_transparency'
    if target_col not in df.columns:
        raise ValueError(f"{target_col} not found in dataframe columns.")
    series = df[target_col].sort_index().asfreq('H').interpolate(limit=24)
    # exogenous
    exog = pd.DataFrame(index=series.index)
    for s in [f'{country}_solar_generation_actual', f'{country}_wind_generation_actual']:
        if s in df.columns:
            exog[s] = df[s].reindex(series.index).interpolate(limit=24)
    X = make_features(series, exog_df=exog).dropna()
    data = pd.concat([series.rename('target'), X], axis=1).dropna()
    split_date = pd.to_datetime('2019-12-31 23:00:00+00:00') if data.index.tzinfo else pd.to_datetime('2019-12-31 23:00:00')
    train = data.loc[data.index<=split_date]
    test = data.loc[data.index>split_date]
    features = [c for c in data.columns if c!='target']
    print(f"DE data prepared. Train size: {len(train)}, Test size: {len(test)}")
    model = train_lgbm(train[features], train['target'], num_round=200)
    pred = model.predict(test[features])
    mae = mean_absolute_error(test['target'].values, pred)
    rmse = math.sqrt(mean_squared_error(test['target'].values, pred))
    # save
    joblib.dump(model, OUTDIR / f'lgbm_{country}_load.joblib')
    pd.concat([test['target'].rename('actual'), pd.Series(pred, index=test.index, name='pred')], axis=1).to_csv(OUTDIR / f'{country}_load_preds.csv')
    # feature importance
    fi = pd.DataFrame({'feature':features, 'importance': model.feature_importance()}).sort_values('importance', ascending=False)
    fi.to_csv(OUTDIR / f'{country}_lgb_feature_importance.csv', index=False)
    # plot (optional)
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,5))
        test['target'].iloc[:168].plot(label='actual', alpha=0.7)
        pd.Series(pred, index=test.index).iloc[:168].plot(label='pred', alpha=0.7)
        plt.legend(); plt.title(f'{country} load: first 168h of test actual vs pred'); plt.tight_layout()
        plt.savefig(OUTDIR / f'{country}_actual_vs_pred_sample.png', dpi=150)
        plt.close()
    except Exception:
        pass
    print(f"{country} LightGBM MAE: {mae:.3f}, RMSE: {rmse:.3f}")
    return model, mae, rmse

In [ ]:
def multi_country_baseline(df, countries=['DE','FR','IT','NL']):
    results = []
    for c in countries:
        col = f'{c}_load_actual_entsoe_transparency'
        if col not in df.columns:
            continue
        ts = df[col].sort_index().asfreq('H').interpolate(limit=24)
        exog = pd.DataFrame(index=ts.index)
        for s in [f'{c}_solar_generation_actual', f'{c}_wind_generation_actual']:
            if s in df.columns:
                exog[s] = df[s].reindex(ts.index).interpolate(limit=24)
        X = make_features(ts, exog_df=exog).dropna()
        data = pd.concat([ts.rename('target'), X], axis=1).dropna()
        if len(data) < 365*24:
            continue
        split_date = pd.to_datetime('2019-12-31 23:00:00+00:00') if data.index.tzinfo else pd.to_datetime('2019-12-31 23:00:00')
        train = data.loc[data.index<=split_date]
        test = data.loc[data.index>split_date]
        features = [col for col in data.columns if col!='target']
        # Persistence
        persist = test['lag_1'].values
        mae_persist = mean_absolute_error(test['target'].values, persist)
        # LightGBM (smaller rounds so it's faster)
        model = train_lgbm(train[features], train['target'], num_round=100)
        pred = model.predict(test[features])
        mae_lgb = mean_absolute_error(test['target'].values, pred)
        # SARIMAX quick
        try:
            sar_train = train['target'][-24*365:] if len(train)>24*365 else train['target']
            sar = SARIMAX(sar_train.values, order=(2,0,0), seasonal_order=(1,0,0,24), enforce_stationarity=False, enforce_invertibility=False)
            res = sar.fit(disp=False, maxiter=50)
            start = len(sar_train)
            end = start + len(test) - 1
            sar_pred = res.predict(start=start, end=end)
            sar_pred = np.array(sar_pred)[:len(test)]
            mae_sar = mean_absolute_error(test['target'].values[:len(sar_pred)], sar_pred)
        except Exception:
            mae_sar = float('nan')
        results.append({'country':c, 'mae_persistence':mae_persist, 'mae_lightgbm':mae_lgb, 'mae_sarimax':mae_sar})
    out = pd.DataFrame(results)
    out.to_csv(OUTDIR / 'multi_country_baseline_results.csv', index=False)
    print("Multi-country baseline results saved to:", OUTDIR / 'multi_country_baseline_results.csv')
    return out